## 강의 2: 시계열 예측 모델링 (Bike Sharing Demand)

- **목표**: 베이스라인 → ML 회귀 → 시계열 회귀 비교, 성능 평가 및 해석
- **데이터**: `train_eda_features.parquet` (강의 1 산출물)
- **메트릭**: RMSLE(대회 기준), RMSE

### 커리큘럼(30분)
1. 데이터 로드/스플릿 고정 (3분)
2. 베이스라인(단순 이동 평균/라그) (5분)
3. 머신러닝 회귀: `RandomForestRegressor` (10분)
4. 시계열 특화: `SARIMAX` or `ExponentialSmoothing` 비교 (8분)
5. 에러 분석과 피처 중요도/잔차 점검 (4분)



In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing

import warnings
warnings.filterwarnings('ignore')

DATA_PATH = os.path.join(os.getcwd(), 'bike-sharing-demand', 'train_eda_features.parquet')
df = pd.read_parquet(DATA_PATH)

# 스플릿 고정
split_time = pd.Timestamp('2012-10-01')
train = df[df['datetime'] < split_time].copy()
valid = df[df['datetime'] >= split_time].copy()

# 메트릭들

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

print(train.shape, valid.shape)

# 베이스라인 1: 최근 24시간 평균
valid_bl = valid.copy()
rolling_window = 24
train['count_rollmean_24h'] = train['count'].rolling(rolling_window).mean()
last_24h_mean = train['count_rollmean_24h'].iloc[-1]
valid_bl['pred_baseline_ma24'] = last_24h_mean
print('Baseline(MA-24h) RMSLE:', rmsle(valid_bl['count'], valid_bl['pred_baseline_ma24']))

# 베이스라인 2: 라그 24시간 값
valid_bl['pred_baseline_lag24'] = valid['count_lag_24']
print('Baseline(Lag-24h) RMSLE:', rmsle(valid_bl['count'], valid_bl['pred_baseline_lag24'].fillna(valid_bl['pred_baseline_ma24'])))

# 머신러닝 피처
target = 'count'
num_features = ['temp','atemp','humidity','windspeed'] + [c for c in df.columns if c.startswith('count_lag_') or c.startswith('count_roll')]
cat_features = ['season','holiday','workingday','weather','year','month','day','hour','weekday']

X_train = train[num_features + cat_features]
y_train = train[target]
X_valid = valid[num_features + cat_features]
y_valid = valid[target]

preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', 'passthrough', num_features)
    ]
)

rf = RandomForestRegressor(n_estimators=300, max_depth=None, n_jobs=-1, random_state=42)
pipe = Pipeline(steps=[('prep', preprocess), ('rf', rf)])

pipe.fit(X_train, y_train)
valid_pred_rf = pipe.predict(X_valid)
print('RandomForest RMSLE:', rmsle(y_valid, valid_pred_rf))

# 시계열 특화: ExponentialSmoothing(일단 일별로 집계 후)
train_d = train.set_index('datetime')['count'].resample('D').sum()
valid_d = valid.set_index('datetime')['count'].resample('D').sum()
model_es = ExponentialSmoothing(train_d, trend='add', seasonal='add', seasonal_periods=7)
fit_es = model_es.fit()
forecast_es = fit_es.forecast(len(valid_d))
print('ExponentialSmoothing (일집계) RMSLE:', rmsle(valid_d.values, np.maximum(forecast_es.values, 0)))

# 결과 집계
results = pd.DataFrame({
    'model': ['baseline_ma24','baseline_lag24','random_forest','exp_smoothing_daily'],
    'rmsle': [
        rmsle(valid_bl['count'], valid_bl['pred_baseline_ma24']),
        rmsle(valid_bl['count'], valid_bl['pred_baseline_lag24'].fillna(valid_bl['pred_baseline_ma24'])),
        rmsle(y_valid, valid_pred_rf),
        rmsle(valid_d.values, np.maximum(forecast_es.values, 0))
    ]
})
print(results.sort_values('rmsle'))

# 피처 중요도 (범주 OHE 포함 파이프에서는 직접 접근 어려움 -> permutation_importance 권장)
from sklearn.inspection import permutation_importance
perm = permutation_importance(pipe, X_valid, y_valid, n_repeats=3, random_state=42, n_jobs=-1)

# 중요도 요약 출력 (파이프 특성명 추출)
feature_names = list(pipe.named_steps['prep'].get_feature_names_out())
fi = pd.Series(perm.importances_mean, index=feature_names).sort_values(ascending=False)[:20]
print(fi)



In [ ]:
# 예측 결과와 실제 비교 시각화 (일/시간)
import matplotlib.pyplot as plt
import seaborn as sns

# 시간 단위 비교 (랜덤포레스트)
valid_vis = valid.copy()
valid_vis['pred_rf'] = valid_pred_rf

plt.figure(figsize=(16,5))
sns.lineplot(x='datetime', y='count', data=valid_vis, label='actual')
sns.lineplot(x='datetime', y='pred_rf', data=valid_vis, label='rf')
plt.title('검증 구간 시간 단위: 실제 vs RF 예측')
plt.tight_layout()
plt.show()

# 일 집계 비교 (ES)
valid_d_vis = valid_d.to_frame('count').copy()
valid_d_vis['pred_es'] = forecast_es.values

plt.figure(figsize=(12,4))
sns.lineplot(data=valid_d_vis)
plt.title('검증 구간 일 단위: 실제 vs ES 예측')
plt.tight_layout()
plt.show()



# Bike Sharing Demand — Time Series Forecasting

## RandomForest vs Prophet vs SARIMAX
이 노트북은 Kaggle **Bike Sharing Demand** 데이터(train.csv)를 사용해
- 머신러닝(RandomForest)
- 시계열 전용(Prophet, *설치된 경우만 실행*)
- 통계 시계열(SARIMAX)

세 가지 접근을 비교합니다. 성능 평가는 **RMSE, MAE**로 진행합니다.

> 그래프는 `matplotlib`만 사용하고, 스타일/색상은 지정하지 않습니다.

## 1. 라이브러리 임포트 및 유틸 함수

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prophet은 환경에 설치되어 있지 않을 수 있어 optional로 처리
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except Exception as e:
    PROPHET_AVAILABLE = False

from statsmodels.tsa.statespace.sarimax import SARIMAX

def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)


ModuleNotFoundError: No module named 'matplotlib'

## 2. 데이터 로드 & 기본 전처리

In [ ]:

# 데이터 로드 (로컬/카글 환경 모두 호환)
import os

def load_bike_train():
    candidate_paths = [
        '/mnt/data/train.csv',
        '/Users/kimjinseok/Desktop/메타코드 강의/bike-sharing-demand/train.csv',
        'bike-sharing-demand/train.csv',
        './train.csv'
    ]
    for p in candidate_paths:
        if os.path.exists(p):
            return pd.read_csv(p)
    raise FileNotFoundError('train.csv 경로를 찾을 수 없습니다. 위 후보 경로를 확인하세요.')

df = load_bike_train()

# datetime 변환
df['datetime'] = pd.to_datetime(df['datetime'])

# 기본 파생변수
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.weekday

# 확인
df.head()


## 3. 빠른 EDA (시각화)

In [ ]:

# 월별 평균 대여량
monthly = df.groupby(['year', 'month'])['count'].mean().reset_index()
monthly['yyyymm'] = monthly['year'].astype(str) + '-' + monthly['month'].astype(str).str.zfill(2)

plt.figure()
plt.plot(monthly['yyyymm'], monthly['count'])
plt.xticks(rotation=45)
plt.title('월별 평균 대여량')
plt.xlabel('월')
plt.ylabel('평균 count')
plt.tight_layout()
plt.show()


In [ ]:

# 시간대별 평균 대여량
hourly = df.groupby('hour')['count'].mean().reset_index()

plt.figure()
plt.plot(hourly['hour'], hourly['count'])
plt.title('시간대별 평균 대여량')
plt.xlabel('Hour')
plt.ylabel('평균 count')
plt.tight_layout()
plt.show()


## 4. 시계열 분할 (Train/Test)
- 시계열의 시간 순서를 보존하기 위해 **마지막 30일**을 테스트로 사용합니다.

In [ ]:

# 마지막 30일 기준으로 train/test 분리
cutoff_date = df['datetime'].max() - pd.Timedelta(days=30)
train_df = df[df['datetime'] <= cutoff_date].copy()
test_df  = df[df['datetime'] > cutoff_date].copy()

train_df.shape, test_df.shape, cutoff_date


## 5. 모델 1 — RandomForestRegressor (머신러닝)

In [ ]:

rf_features = ['year', 'month', 'day', 'hour', 'weekday', 'season', 'holiday', 'workingday', 'weather', 'temp', 'humidity', 'windspeed']
target = 'count'

X_train = train_df[rf_features]
y_train = train_df[target]
X_test  = test_df[rf_features]
y_test  = test_df[target]

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

rf_rmse = rmse(y_test, pred_rf)
rf_mae  = mae(y_test, pred_rf)

rf_rmse, rf_mae


In [ ]:

# 피처 중요도 시각화
importances = pd.Series(rf.feature_importances_, index=rf_features).sort_values(ascending=False)

plt.figure()
importances.plot(kind='bar')
plt.title('RandomForest Feature Importances')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()


In [ ]:

# 예측 vs 실제
plt.figure()
plt.plot(test_df['datetime'].values, y_test.values, label='Actual')
plt.plot(test_df['datetime'].values, pred_rf, label='RF Pred')
plt.title('RandomForest: 실제 vs 예측')
plt.xlabel('datetime')
plt.ylabel('count')
plt.legend()
plt.tight_layout()
plt.show()


## 5b. 쉬운 베이스라인 — 복잡한 모델 전에 기준선부터
- **Naive-24h**: 전일 같은 시간의 수요를 그대로 예측
- **Hour-Mean**: 학습기간의 시간대별 평균을 예측치로 사용


In [ ]:
# 베이스라인 계산
s_all = df.set_index('datetime')['count'].asfreq('H')
# 전일 같은 시간
pred_naive = []
for ts in test_df['datetime']:
    prev = ts - pd.Timedelta(hours=24)
    val = s_all.get(prev, np.nan)
    pred_naive.append(val)

pred_naive = np.array(pred_naive)
bl24_valid = ~np.isnan(pred_naive)
bl24_rmse = rmse(y_test.values[bl24_valid], pred_naive[bl24_valid])
bl24_mae  = mae(y_test.values[bl24_valid], pred_naive[bl24_valid])

# 시간대 평균
hour_mean = train_df.groupby('hour')['count'].mean()
pred_hmean = test_df['hour'].map(hour_mean).values
hmean_rmse = rmse(y_test.values, pred_hmean)
hmean_mae  = mae(y_test.values, pred_hmean)

print('Baseline(Naive-24h) RMSE/MAE:', round(bl24_rmse,2), round(bl24_mae,2))
print('Baseline(Hour-Mean) RMSE/MAE:', round(hmean_rmse,2), round(hmean_mae,2))


## 5c. 간단 특징 만들기 — 지연(lag)과 이동평균(roll)
- 예측 시점 이전 정보만 사용해 누출을 방지합니다.
- 핵심: `lag_1`, `lag_24`, `roll3`, `roll24`.


In [ ]:
# 전체 데이터에서 순서대로 피처 생성 후 train/test 분리
full = df.sort_values('datetime').set_index('datetime').copy()
full['lag_1']  = full['count'].shift(1)
full['lag_24'] = full['count'].shift(24)
full['roll3']  = full['count'].shift(1).rolling(3).mean()
full['roll24'] = full['count'].shift(1).rolling(24).mean()

# 분할 동일 기준 사용
train_fe = full.loc[:cutoff_date].dropna().copy()
test_fe  = full.loc[cutoff_date + pd.Timedelta(seconds=0):].dropna().copy()

# 원본 공변량(시간/기상) 인덱스 정렬 후 조인
exog_cols = ['hour','temp','humidity','season','holiday','workingday','weather','windspeed','atemp']
exog = df.set_index('datetime')[exog_cols]

# 선형회귀용 단순 피처
Xtr_lr = pd.concat([
    train_fe[['lag_24']],
    exog.loc[train_fe.index, ['hour','temp','humidity']]
], axis=1)
ytr    = train_fe['count']
Xte_lr = pd.concat([
    test_fe[['lag_24']],
    exog.loc[test_fe.index, ['hour','temp','humidity']]
], axis=1)
yte    = df.set_index('datetime').loc[test_fe.index, 'count']

# RF(+lag)용 확장 피처
Xtr_rf = pd.concat([
    train_fe[['lag_1','lag_24','roll3','roll24']],
    exog.loc[train_fe.index, exog_cols]
], axis=1)
Xte_rf = pd.concat([
    test_fe[['lag_1','lag_24','roll3','roll24']],
    exog.loc[test_fe.index, exog_cols]
], axis=1)


## 5d. 해석 쉬운 선형회귀 + 간단 랜덤포레스트(피처 포함)
- 선형회귀: `lag_24`와 기상/시간 정보를 이용한 단순 예측
- RF(+lag): 트리 기반으로 비선형 패턴까지 포착


In [ ]:
from sklearn.linear_model import LinearRegression

# 선형회귀
lr = LinearRegression()
lr.fit(Xtr_lr, ytr)
pred_lr = lr.predict(Xte_lr)
lr_rmse = rmse(yte.values, pred_lr)
lr_mae  = mae(yte.values, pred_lr)
print('LinearRegression RMSE/MAE:', round(lr_rmse,2), round(lr_mae,2))

# lag 포함 RF (가벼운 사양)
rf2 = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf2.fit(Xtr_rf, ytr)
pred_rf2 = rf2.predict(Xte_rf)
rf2_rmse = rmse(yte.values, pred_rf2)
rf2_mae  = mae(yte.values, pred_rf2)
print('RF(+lag) RMSE/MAE:', round(rf2_rmse,2), round(rf2_mae,2))


In [ ]:
# 간단 비교 바차트
labels = ['BL-24h','BL-HourMean','RF(orig)','LR(+lag)','RF(+lag)','SARIMAX'] + (['Prophet'] if 'prophet_rmse' in globals() or 'prophet_rmse' in locals() else [])
values = [bl24_rmse, hmean_rmse, rf_rmse, lr_rmse, rf2_rmse, sarimax_rmse] + ([prophet_rmse] if 'prophet_rmse' in globals() or 'prophet_rmse' in locals() else [])

plt.figure()
plt.bar(range(len(labels)), values)
plt.xticks(range(len(labels)), labels, rotation=45)
plt.ylabel('RMSE (낮을수록 좋음)')
plt.title('모델/베이스라인 성능 비교')
plt.tight_layout()
plt.show()


In [ ]:
# 확장 비교 표
results2 = {
    'BL-24h': {'RMSE': bl24_rmse, 'MAE': bl24_mae},
    'BL-HourMean': {'RMSE': hmean_rmse, 'MAE': hmean_mae},
    'RandomForest(orig)': {'RMSE': rf_rmse, 'MAE': rf_mae},
    'LinearRegression(+lag)': {'RMSE': lr_rmse, 'MAE': lr_mae},
    'RandomForest(+lag)': {'RMSE': rf2_rmse, 'MAE': rf2_mae},
    'SARIMAX': {'RMSE': sarimax_rmse, 'MAE': sarimax_mae}
}
if PROPHET_AVAILABLE:
    results2['Prophet'] = {'RMSE': prophet_rmse, 'MAE': prophet_mae}

res_df2 = pd.DataFrame(results2).T
res_df2


### 강의 전달 팁
- 먼저 베이스라인으로 문제 감을 잡은 뒤, 간단 특징과 선형회귀로 “왜 그런지”를 설명합니다.
- 이후 RandomForest(+lag)로 성능을 올리고, SARIMAX/Prophet은 맛보기 비교 정도로만 다룹니다.
- 실무 포인트: 시간 누출 방지(shift), 시간대 평균 같은 운영 친화적 기준선을 항상 함께 제시하세요.


## 6. 모델 2 — Prophet (시계열 전용, 설치된 경우만 실행)

In [ ]:

if PROPHET_AVAILABLE:
    # Prophet은 ds, y 컬럼 사용
    train_p = train_df[['datetime', 'count']].rename(columns={'datetime': 'ds', 'count': 'y'})
    test_p  = test_df[['datetime', 'count']].rename(columns={'datetime': 'ds', 'count': 'y'})
    
    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True
    )
    m.fit(train_p)
    
    # 테스트 기간 동안 예측
    future = pd.DataFrame({'ds': test_p['ds']})
    forecast = m.predict(future)
    
    # Prophet 예측 결과와 실제 매칭
    pred_prophet = forecast['yhat'].values
    prophet_rmse = rmse(test_p['y'].values, pred_prophet)
    prophet_mae  = mae(test_p['y'].values, pred_prophet)
    
    print('Prophet RMSE, MAE:', prophet_rmse, prophet_mae)
    
    # 시각화
    plt.figure()
    plt.plot(test_p['ds'].values, test_p['y'].values, label='Actual')
    plt.plot(test_p['ds'].values, pred_prophet, label='Prophet Pred')
    plt.title('Prophet: 실제 vs 예측')
    plt.xlabel('datetime')
    plt.ylabel('count')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print('Prophet이 설치되어 있지 않아 이 섹션을 건너뜁니다. (pip install prophet 후 재실행)')


## 7. 모델 3 — SARIMAX (통계 시계열)
- 간단한 파라미터로 예시를 실행합니다. (시즌 주기: 24시간)

In [ ]:

# 시계열 인덱스 구성
ts_train = train_df.set_index('datetime')['count'].asfreq('H')
ts_test  = test_df.set_index('datetime')['count'].asfreq('H')

# 결측이 있다면 0으로 채움(간단 처리)
ts_train = ts_train.fillna(0)
ts_test  = ts_test.fillna(0)

# 간단한 사양: (p,d,q)=(1,1,1), (P,D,Q,m)=(1,1,1,24)
# 학습 시간 단축을 위해 enforce_stationarity/enforce_invertibility=False
model = SARIMAX(ts_train,
                order=(1,1,1),
                seasonal_order=(1,1,1,24),
                enforce_stationarity=False,
                enforce_invertibility=False)
res = model.fit(disp=False)

# 테스트 구간 예측
pred_sarimax = res.get_forecast(steps=len(ts_test)).predicted_mean

sarimax_rmse = rmse(ts_test.values, pred_sarimax.values)
sarimax_mae  = mae(ts_test.values, pred_sarimax.values)

sarimax_rmse, sarimax_mae


In [ ]:

plt.figure()
plt.plot(ts_test.index, ts_test.values, label='Actual')
plt.plot(ts_test.index, pred_sarimax.values, label='SARIMAX Pred')
plt.title('SARIMAX: 실제 vs 예측')
plt.xlabel('datetime')
plt.ylabel('count')
plt.legend()
plt.tight_layout()
plt.show()


## 8. 성능 비교 (RMSE / MAE)

In [ ]:

results = {
    'RandomForest': {'RMSE': rf_rmse, 'MAE': rf_mae},
    'SARIMAX': {'RMSE': sarimax_rmse, 'MAE': sarimax_mae}
}
if PROPHET_AVAILABLE:
    results['Prophet'] = {'RMSE': prophet_rmse, 'MAE': prophet_mae}

res_df = pd.DataFrame(results).T
res_df


In [ ]:

from caas_jupyter_tools import display_dataframe_to_user
display_dataframe_to_user("Model Performance (RMSE/MAE)", res_df)
